In [80]:
from   ast import literal_eval
from   collections import Counter
from   geopy import distance
from   itertools import pairwise
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import random
from   scipy.stats import permutation_test
import seaborn

## Load derived data

In [36]:
data_dir = os.path.join('..', 'data')
derived_dir = os.path.join(data_dir, 'derived')

def string_to_list(x):
    lst = literal_eval(x.replace(', nan', "', ZZZZ'").replace('[nan, ', "'['").replace(', nan]', "']'"))
    return [i for i in lst if i != 'ZZZZ']

df = pd.read_csv(
    os.path.join(derived_dir, 'CONLIT_CharData_AP_3.csv.gz'), 
    index_col='book_id',
    converters={
        'gpe_places': string_to_list,
        'nongpe_places': string_to_list,
        'all_places': string_to_list,
        'gpe_sequences': string_to_list
    },
)
df.fillna('')
df.head()

,X,char_id,char_count,inf_gender,gpe_places,num_gpe_places,nongpe_places,num_nongpe_places,all_places,num_all_places,...,char_count_norm,nongpe_places_cleaned,nongpe_places_total,gpe_places_total,ttr_nongpe,ttr_gpe,avg_Distance_GPE,non_gpe_ratio,avg_Distance_GPE_Tokens,deixis_count_perplace
book_id,,,,,,,,,,,,,,,,,,,,,
[Heist Society 2] Uncommon Criminals - Ally Carter,1,138,3238,she/her,"[Paraguay, Paraguay, London, New York City, mi...",15,"[there, there, there, the museum, the museum, ...",103,"[there, there, there, Paraguay, Paraguay, the ...",118,...,0.040278,"['', '', '', 'the museum', 'the museum', 'the ...",238,32,0.432773,0.468750,1664.751474,7.437500,0.662654,0.084034
"2001_2011_Wilson,RobertCharles_TheChronoliths_SF",2,0,2742,she/her,"[Bangkok, Minneapolis / St. Paul, Chumphon, Ch...",22,"[our household, a gas station hawng nam, the c...",122,"[our household, a gas station hawng nam, the c...",143,...,0.026562,"['our household', 'a gas station hawng nam', '...",192,62,0.635417,0.354839,2895.067535,3.096774,1.738779,0.135417
"2001_Martel,Yann_LifeofPi_BS",3,0,4754,he/him/his,"[Canada, India, India, India, India, India, Ca...",15,"[the north, the rich , noisy , functioning mad...",142,"[Canada, India, the north, India, India, the r...",157,...,0.038202,"['the north', 'the rich , noisy , functioning ...",282,54,0.503546,0.277778,3695.789883,5.222222,1.603702,0.031915
"2002_2011_Anderson,MT_Feed_SF",4,0,2862,she/her,"[Lonely, School, School, Switzerland, Io, Amer...",5,"[the craters, a crummy hotel, here, here, the ...",68,"[the craters, a crummy hotel, here, here, the ...",72,...,0.042669,"['the craters', 'a crummy hotel', '', '', 'the...",143,7,0.475524,0.714286,713.296839,20.428571,0.074440,0.097902
"2002_Baker,Jo_Offcomer_CT",5,120,4723,she/her,"[Belfast, Belfast, Belfast, Belfast, Belfast, ...",11,"[the bath, the bath, the bath, it, the bath, t...",162,"[the bath, the bath, the bath, it, the bath, t...",172,...,0.053645,"['the bath', 'the bath', 'the bath', '', 'the ...",400,36,0.405000,0.305556,524.871736,11.111111,0.214618,0.075000


In [4]:
df.columns

Index(['X', 'char_id', 'char_count', 'inf_gender', 'gpe_places',
       'num_gpe_places', 'nongpe_places', 'num_nongpe_places', 'all_places',
       'num_all_places', 'gpe_sequences', 'dist_miles', 'char_rank',
       'num_words', 'Category', 'Genre', 'Tokens', 'num_gpe_places_norm',
       'num_nongpe_places_norm', 'num_gpe_places_norm_byCharacter',
       'num_nongpe_places_norm_byCharacter', 'char_count_norm',
       'nongpe_places_cleaned', 'nongpe_places_total', 'gpe_places_total',
       'ttr_nongpe', 'ttr_gpe', 'avg_Distance_GPE', 'non_gpe_ratio',
       'avg_Distance_GPE_Tokens', 'deixis_count_perplace'],
      dtype='object')

In [5]:
df.Category.value_counts()

Category
FIC    1934
NON     820
Name: count, dtype: int64

In [6]:
df.Genre.value_counts()

Genre
NYT     419
PW      258
BS      249
MY      234
MEM     229
SF      223
ROM     208
HIST    205
MIX     193
BIO     193
YA      177
MID     166
Name: count, dtype: int64

## Feature differences by facet

In [12]:
df.nongpe_places_total

book_id
[Heist Society 2] Uncommon Criminals - Ally Carter    238
2001_2011_Wilson,RobertCharles_TheChronoliths_SF      192
2001_Martel,Yann_LifeofPi_BS                          282
2002_2011_Anderson,MT_Feed_SF                         143
2002_Baker,Jo_Offcomer_CT                             400
                                                     ... 
Woman Behind the New Deal, The - Kirstin Downey       537
Woman of No Importance, A - Sonia Purnell             445
Woodrow Wilson - John Milton Cooper, Jr_              241
You Never Forget Your First - Alexis Coe               18
Young Stalin - Montefiore, Simon Sebag                369
Name: nongpe_places_total, Length: 2754, dtype: int64

In [15]:
facet_by = ['inf_gender', 'Category', 'Genre']

cols_to_test = [
    'char_count', 
    'num_gpe_places', 
    'num_nongpe_places',
    'num_all_places', 
    'dist_miles', 
    'Tokens', 
    'num_gpe_places_norm',
    'num_nongpe_places_norm', 
    'num_gpe_places_norm_byCharacter',
    'num_nongpe_places_norm_byCharacter', 
    'char_count_norm',
    'nongpe_places_total', 
    'gpe_places_total',
    'ttr_nongpe', 
    'ttr_gpe', 
    'avg_Distance_GPE', 
    'non_gpe_ratio',
    'avg_Distance_GPE_Tokens', 
    'deixis_count_perplace'
]

def significance_label(pvalue, levels=[0.05, 0.01, 0.001]):
    if pvalue < levels[2]:
        return '***'
    elif pvalue < levels[1]:
        return '**'
    elif pvalue < levels[0]:
        return '*'
    else: 
        return ''

def diff_means(x, y, axis):
    return np.mean(x, axis=axis) - np.mean(y, axis=axis)

def compare_facets(df, facets, cols, statistic=diff_means):
    '''Compare statistic in cols of df via permutation. Display results, return nothing.'''
    for facet in facets:
        print(f"\n==============\nFacet: {facet}\n==============")
        df = df.dropna(subset=[facet])
        for level in df[facet].unique():
            data = (df.loc[df[facet]==level, cols], df.loc[~(df[facet]==level), cols])
            level_mean = np.mean(data[0], axis=0)
            other_mean = np.mean(data[1], axis=0)
            output = pd.DataFrame(level_mean, columns=[level]).join(pd.DataFrame(other_mean, columns=['others']))
            res = permutation_test(data, statistic)
            output['diff'] = res.statistic
            output['p'] = res.pvalue
            output['sig'] = output['p'].apply(significance_label)
            display(output)

In [16]:
compare_facets(df, facet_by, cols_to_test, diff_means)


Facet: inf_gender


,she/her,others,diff,p,sig
char_count,3752.279775,3844.495428,-92.215652,0.3940,
num_gpe_places,16.356180,21.814954,-5.458775,0.0002,***
num_nongpe_places,122.757303,123.285637,-0.528334,0.8636,
num_all_places,138.620225,144.524476,-5.904251,0.0996,
dist_miles,56148.685011,79583.242665,-23434.557654,0.0002,***
Tokens,119516.989888,131900.656267,-12383.666379,0.0002,***
num_gpe_places_norm,0.000141,0.000170,-0.000029,0.0002,***
num_nongpe_places_norm,0.001127,0.001048,0.000079,0.0012,**
num_gpe_places_norm_byCharacter,0.005224,0.009569,-0.004345,0.0002,***
num_nongpe_places_norm_byCharacter,0.036178,0.037657,-0.001479,0.0400,*


,he/him/his,others,diff,p,sig
char_count,4012.726801,3475.705128,537.021673,0.0002,***
num_gpe_places,22.025360,16.663708,5.361652,0.0002,***
num_nongpe_places,127.342939,115.879684,11.463255,0.0002,***
num_all_places,148.785591,132.051282,16.734309,0.0002,***
dist_miles,80558.158976,57346.323422,23211.835554,0.0002,***
Tokens,133472.337752,118342.144970,15130.192782,0.0002,***
num_gpe_places_norm,0.000169,0.000146,0.000023,0.0002,***
num_nongpe_places_norm,0.001073,0.001074,-0.000001,0.9472,
num_gpe_places_norm_byCharacter,0.008396,0.007762,0.000634,0.1794,
num_nongpe_places_norm_byCharacter,0.036880,0.037687,-0.000807,0.2436,


,they/them/their,others,diff,p,sig
char_count,1490.612903,3924.422857,-2433.809954,0.0002,***
num_gpe_places,18.870968,20.103238,-1.232270,0.5624,
num_nongpe_places,66.516129,125.788190,-59.272061,0.0002,***
num_all_places,84.903226,145.339048,-60.435822,0.0002,***
dist_miles,65942.276537,72282.184946,-6339.908409,0.5872,
Tokens,109909.790323,128740.810286,-18831.019963,0.0010,**
num_gpe_places_norm,0.000183,0.000160,0.000024,0.0876,
num_nongpe_places_norm,0.000695,0.001091,-0.000396,0.0002,***
num_gpe_places_norm_byCharacter,0.025981,0.007321,0.018660,0.0002,***
num_nongpe_places_norm_byCharacter,0.048523,0.036642,0.011881,0.0002,***



Facet: Category


...

,PW,others,diff,p,sig
char_count,3177.618677,3880.336276,-702.717599,0.0002,***
num_gpe_places,14.801556,20.588684,-5.787127,0.0002,***
num_nongpe_places,121.428016,123.288523,-1.860508,0.7030,
num_all_places,135.782101,143.317416,-7.535315,0.1788,
dist_miles,52226.899827,74035.017864,-21808.118037,0.0016,**
Tokens,122261.315175,128472.023676,-6210.708501,0.1470,
num_gpe_places_norm,0.000134,0.000163,-0.000029,0.0010,**
num_nongpe_places_norm,0.001108,0.001070,0.000038,0.3422,
num_gpe_places_norm_byCharacter,0.005194,0.008469,-0.003275,0.0002,***
num_nongpe_places_norm_byCharacter,0.042319,0.036648,0.005672,0.0002,***


,MIX,others,diff,p,sig
char_count,1869.167539,3959.904222,-2090.736683,0.0002,***
num_gpe_places,25.450262,19.644253,5.806008,0.0012,**
num_nongpe_places,84.507853,125.997263,-41.489410,0.0002,***
num_all_places,109.403141,145.092651,-35.689509,0.0002,***
dist_miles,103312.422699,69657.898764,33654.523935,0.0012,**
Tokens,126088.335079,128026.023847,-1937.688768,0.7044,
num_gpe_places_norm,0.000204,0.000157,0.000047,0.0002,***
num_nongpe_places_norm,0.000745,0.001098,-0.000352,0.0002,***
num_gpe_places_norm_byCharacter,0.018654,0.007379,0.011275,0.0002,***
num_nongpe_places_norm_byCharacter,0.046469,0.036484,0.009985,0.0002,***


,MEM,others,diff,p,sig
char_count,4260.253275,3774.146032,486.107243,0.0120,*
num_gpe_places,37.877729,18.427381,19.450348,0.0002,***
num_nongpe_places,170.908297,118.771429,52.136868,0.0002,***
num_all_places,207.737991,136.694841,71.043150,0.0002,***
dist_miles,159239.594297,64068.139159,95171.455138,0.0002,***
Tokens,105611.161572,129916.065476,-24304.903904,0.0002,***
num_gpe_places_norm,0.000355,0.000143,0.000212,0.0002,***
num_nongpe_places_norm,0.001666,0.001019,0.000646,0.0002,***
num_gpe_places_norm_byCharacter,0.009739,0.008019,0.001720,0.0490,*
num_nongpe_places_norm_byCharacter,0.041595,0.036777,0.004819,0.0002,***


,MID,others,diff,p,sig
char_count,3103.355422,3860.351916,-756.996495,0.0002,***
num_gpe_places,7.024096,20.884630,-13.860534,0.0002,***
num_nongpe_places,94.024096,124.984127,-30.960031,0.0002,***
num_all_places,100.753012,145.303136,-44.550124,0.0002,***
dist_miles,16320.877678,75574.259419,-59253.381741,0.0002,***
Tokens,84156.487952,130702.076655,-46545.588703,0.0002,***
num_gpe_places_norm,0.000089,0.000165,-0.000076,0.0002,***
num_nongpe_places_norm,0.001243,0.001062,0.000181,0.0002,***
num_gpe_places_norm_byCharacter,0.002412,0.008532,-0.006120,0.0002,***
num_nongpe_places_norm_byCharacter,0.031853,0.037520,-0.005667,0.0002,***


,BIO,others,diff,p,sig
char_count,4291.549223,3778.629499,512.919724,0.0150,*
num_gpe_places,59.735751,17.050861,42.684891,0.0002,***
num_nongpe_places,175.652850,119.147496,56.505354,0.0002,***
num_all_places,233.875648,135.721831,98.153817,0.0002,***
dist_miles,252167.099482,58391.755702,193775.343780,0.0002,***
Tokens,196532.989637,122708.362285,73824.627352,0.0002,***
num_gpe_places_norm,0.000331,0.000148,0.000184,0.0002,***
num_nongpe_places_norm,0.001036,0.001076,-0.000040,0.3754,
num_gpe_places_norm_byCharacter,0.015934,0.007576,0.008359,0.0002,***
num_nongpe_places_norm_byCharacter,0.046214,0.036496,0.009718,0.0002,***


## GPE distances

In [39]:
gpe_cols = ['gpe_places', 'gpe_sequences']

In [69]:
gpe_counts = Counter()
for i in df.gpe_places:
    gpe_counts.update(i)
gpe_counts.most_common(20)

[('New York', 6683),
 ('London', 4367),
 ('Paris', 3673),
 ('America', 3477),
 ('Washington', 2412),
 ('England', 2087),
 ('California', 2064),
 ('Chicago', 2000),
 ('Boston', 1624),
 ('France', 1560),
 ('Rome', 1304),
 ('San Francisco', 1247),
 ('Los Angeles', 1247),
 ('Texas', 1047),
 ('New York City', 1046),
 ('Europe', 952),
 ('Berlin', 938),
 ('Manhattan', 908),
 ('Philadelphia', 862),
 ('Virginia', 854)]

### Geonames

It **looks** like `gpe_places` is the observed sequence of GPEs in the source text, while `gpe_sequences` de-dupes (naïvely) immediate repetitions (but not repetitions with an intervening place). **We should confirm this with Sandeep if we're not sure.** 

In [40]:
gn_col_names = [
    'geonameid',
    'name',
    'asciiname',
    'alternatenames',
    'lat',
    'lon',
    'feature_class',
    'feature_code',
    'country_code',
    'cc2',
    'admin1_code',
    'admin2_code',
    'admin3_code',
    'admin4_code',
    'population',
    'elevation',
    'dem',
    'timezone',
    'mod_date'
]
gn = pd.read_csv(
    os.path.join(data_dir, 'geonames', 'allCountries.zip'), 
    sep='\t',
    names=gn_col_names,
    index_col=0,
    low_memory=False
)
gn.sample(5)

/var/folders/7w/9w42413d6yq9tdnnb2kd8wgc0000gn/T/ipykernel_42651/2729711462.py:22: DtypeWarning: Columns (9,10,11,12,13) have mixed types. Specify dtype option on import or set low_memory=False.
  gn = pd.read_csv(


,name,asciiname,alternatenames,lat,lon,feature_class,feature_code,country_code,cc2,admin1_code,admin2_code,admin3_code,admin4_code,population,elevation,dem,timezone,mod_date
geonameid,,,,,,,,,,,,,,,,,,
11696518,Zhongmin Village,Zhongmin Village,"Zhongmin,Zhongmin Village,zhong min li,中民里",25.06006,121.49503,A,ADM4,TW,NaN,3.0,TPQ,F05,65000020029.0,0,NaN,11,Asia/Taipei,2017-09-23
923902,Simeon Mvula,Simeon Mvula,NaN,-11.86834,33.57762,P,PPL,MW,NaN,N,15.0,NaN,NaN,0,NaN,1294,Africa/Blantyre,2011-06-06
5187252,Doylestown Country Club,Doylestown Country Club,NaN,40.30538,-75.11739,S,RSRT,US,NaN,PA,17,19792.0,NaN,0,91.0,97,America/New_York,2017-05-23
10834884,Chamārtola,Chamartola,NaN,24.50256,84.02197,P,PPL,IN,NaN,38.0,358.0,NaN,NaN,0,NaN,163,Asia/Kolkata,2016-02-11
7188992,Marquam United Methodist Church,Marquam United Methodist Church,NaN,45.07222,-122.68806,S,CH,US,NaN,OR,5,NaN,NaN,0,91.0,92,America/Los_Angeles,2010-02-05


In [41]:
hierarchy = pd.read_csv(
    os.path.join(data_dir, 'geonames', 'hierarchy.zip'), 
    sep='\t',
    names=['parent', 'child', 'type']
)
hierarchy

,parent,child,type
0,6295630,6255146,ADM
1,6295630,6255152,ADM
2,6295630,6255147,ADM
3,6295630,6255148,ADM
4,6295630,6255149,ADM
...,...,...,...
492650,648605,12557021,NaN
492651,12558102,3923974,NaN
492652,1680578,1712681,NaN
492653,3996063,8581707,NaN


In [46]:
gn.loc[[6255146, 6295630]]

,name,asciiname,alternatenames,lat,lon,feature_class,feature_code,country_code,cc2,admin1_code,admin2_code,admin3_code,admin4_code,population,elevation,dem,timezone,mod_date
geonameid,,,,,,,,,,,,,,,,,,
6255146,Africa,Africa,"Aafrika,Affrica,Africa,Afrihkka,Afrihkká,Afrik...",7.1881,21.09375,L,CONT,NaN,"AO,BF,BI,BJ,BW,CD,CF,CG,CI,CM,CV,DJ,DZ,EG,ER,E...",NaN,NaN,NaN,NaN,1031833000,NaN,592,Africa/Bangui,2019-08-12
6295630,Earth,Earth,"Aarde,Earth,Er shary,Erde,Foeld,Föld,Ge,Globe,...",0.0000,0.00000,L,AREA,NaN,NaN,NaN,NaN,NaN,NaN,6814400000,NaN,-9999,NaN,2023-04-16


In [18]:
feature_codes = pd.read_csv(
    os.path.join(data_dir, 'geonames', 'featureCodes_en.txt'),
    sep='\t',
    names=['feature_code', 'feature_name', 'feature_description']
)
feature_codes

,feature_code,feature_name,feature_description
0,A.ADM1,first-order administrative division,a primary administrative division of a country...
1,A.ADM1H,historical first-order administrative division,a former first-order administrative division
2,A.ADM2,second-order administrative division,a subdivision of a first-order administrative ...
3,A.ADM2H,historical second-order administrative division,a former second-order administrative division
4,A.ADM3,third-order administrative division,a subdivision of a second-order administrative...
...,...,...,...
676,V.TREE,tree(s),a conspicuous tree used as a landmark
677,V.TUND,tundra,"a marshy, treeless, high latitude plain, domin..."
678,V.VIN,vineyard,a planting of grapevines
679,V.VINS,vineyards,plantings of grapevines


In [12]:
gn.loc[gn.feature_code=='ADM1'].sample(5)

,name,asciiname,alternatenames,lat,lon,feature_class,feature_code,country_code,cc2,admin1_code,admin2_code,admin3_code,admin4_code,population,elevation,dem,timezone,mod_date
geonameid,,,,,,,,,,,,,,,,,,
4566106,Luquillo Municipio,Luquillo Municipio,"LUQ,Luquillo",18.33967,-65.72461,A,ADM1,PR,NaN,89.0,NaN,NaN,NaN,20068,125.0,111,America/Puerto_Rico,2019-04-14
3983035,Estado de Sinaloa,Estado de Sinaloa,"Estado de Sinaloa,SIN,Sinaloa,shinaroa zhou,xi...",25.00000,-107.50000,A,ADM1,MX,NaN,25,NaN,NaN,NaN,3026943,NaN,139,America/Mazatlan,2022-12-05
142550,West Azerbaijan Province,West Azerbaijan Province,"Azarbaijan Gharbi,Azarbayjan-e Baxtari,Azarbay...",37.66667,45.00000,A,ADM1,IR,NaN,1.0,NaN,NaN,NaN,3265219,NaN,1350,Asia/Tehran,2021-01-13
1855608,Nara-ken,Nara-ken,"Nara,Nara Prefecture,nai liang,nai liang xian,...",34.68525,135.83289,A,ADM1,JP,NaN,28,NaN,NaN,NaN,1321805,NaN,96,Asia/Tokyo,2023-07-18
460311,Dobeles novads,Dobeles novads,"Dobel'skiy Rayon,Dobele Municipality,Dobeles R...",56.58333,23.16667,A,ADM1,LV,NaN,08,NaN,NaN,NaN,28517,NaN,88,Europe/Riga,2023-07-08


In [25]:
gn.loc[(gn.name=='Paris') & (gn.population>0)].sort_values(by='population', ascending=False)

,name,asciiname,alternatenames,lat,lon,feature_class,feature_code,country_code,cc2,admin1_code,admin2_code,admin3_code,admin4_code,population,elevation,dem,timezone,mod_date
geonameid,,,,,,,,,,,,,,,,,,
2988506,Paris,Paris,"Paras,Parigi,Paris,París,Páras",48.85339,2.34864,A,ADM3,FR,NaN,11,75,751,NaN,4380654,NaN,35,Europe/Paris,2022-03-27
6455259,Paris,Paris,"75056,Paris",48.86000,2.34444,A,ADM4,FR,NaN,11,75,751,75056,2190327,NaN,54,Europe/Paris,2019-03-23
2968815,Paris,Paris,"Departement de Paris,Département de Paris,Pari...",48.85340,2.34860,A,ADM2,FR,NaN,11,75,NaN,NaN,2165423,NaN,35,Europe/Paris,2022-03-10
2988507,Paris,Paris,"Baariis,Bahliz,Baris,Lungsod ng Paris,Lutece,L...",48.85341,2.34880,P,PPLC,FR,NaN,11,75,751,75056,2138551,NaN,42,Europe/Paris,2023-09-22
4717560,Paris,Paris,"PRX,Paris,Parizh,barys,parys tgzas,Париж,Пари...",33.66094,-95.55551,P,PPLA2,US,NaN,TX,277,NaN,NaN,24782,183.0,190,America/Chicago,2017-03-09
6942553,Paris,Paris,پیرس، انٹاریو,43.20000,-80.38333,P,PPL,CA,NaN,8.0,NaN,NaN,NaN,12310,NaN,255,America/Toronto,2021-12-21
4647963,Paris,Paris,"PHT,Paris,Parizh,barys,parys tnsy,Париж,Парис...",36.30200,-88.32671,P,PPLA2,US,NaN,TN,79,NaN,NaN,10150,157.0,158,America/Chicago,2017-03-09
4303602,Paris,Paris,"Paris,Parizh,ba li,barys,parys kntaky,Париж,П...",38.20980,-84.25299,P,PPLA2,US,NaN,KY,17,NaN,NaN,9870,257.0,258,America/New_York,2017-03-09
4246659,Paris,Paris,"Parizh,Peris,barys,parys aylynwy,Париж,Перис,...",39.61115,-87.69614,P,PPLA2,US,NaN,IL,45,57641.0,NaN,8432,220.0,223,America/Chicago,2017-05-23


In [72]:
gn.loc[gn.name=='London'].sort_values(by='population', ascending=False)

,name,asciiname,alternatenames,lat,lon,feature_class,feature_code,country_code,cc2,admin1_code,admin2_code,admin3_code,admin4_code,population,elevation,dem,timezone,mod_date
geonameid,,,,,,,,,,,,,,,,,,
2643743,London,London,"ILondon,LON,Lakana,Landan,Landen,Ljondan,Llund...",51.50853,-0.12574,P,PPLC,GB,,ENG,GLA,,,8961989,,25,Europe/London,2022-03-09
11591955,London,London,London,51.49227,-0.30864,L,RGN,GB,,ENG,,,,8787892,,16,Europe/London,2023-09-28
6058560,London,London,"Landona,London,Londonas,Londono,YXU,leondeon,l...",42.98339,-81.23304,P,PPL,CA,,8.0,,,,346765,,252,America/Toronto,2012-08-19
4517009,London,London,"Landon,Limerick,London,New London,lndn,lndn a...",39.88645,-83.44825,P,PPLA2,US,,OH,97.0,44674.0,,10060,321.0,321,America/New_York,2017-05-23
4298960,London,London,"LOZ,Landon,London,Riceton,lndn,lndn kntaky,lu...",37.12898,-84.08326,P,PPLA2,US,,KY,125,,,8126,378.0,379,America/New_York,2017-03-09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4637795,London,London,,35.86982,-83.00209,P,PPL,US,,TN,29,,,0,443.0,451,America/New_York,2006-01-15
4707415,London,London,"London,Norfolk",32.23099,-94.94438,P,PPL,US,,TX,401,,,0,165.0,167,America/Chicago,2006-01-15
6058559,London,London,,43.08339,-81.29975,L,AREA,CA,,8.0,,,,0,,281,America/Toronto,2006-01-18


### Wilkens geo data

In [147]:
wi = pd.read_csv(
    os.path.join('..', '..', 'toponyms', 'geo.tsv.gz'),
    sep='\t',
    low_memory=False
)
wi = wi.loc[wi.lang=='en']

In [148]:
wi.head()

,text_string,lang,occurs,place_id,formatted_address,location_type,country_long,country_short,admin_1_long,admin_1_short,...,route,post_code,natural_feature,point_of_interest,colloquial_area,continent,other,lat,lon,ignore
0,bow london uk,en,NaN,ChIJWdIbuTkddkgRmL6MjjLkU_M,"Bow, London, UK",neighborhood,United Kingdom,GB,England,England,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,51.532089,-0.030820,NaN
1,highbury london uk,en,NaN,ChIJU2SdvHkbdkgREQdR0C6uDiY,"Highbury, London, UK",neighborhood,United Kingdom,GB,England,England,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,51.556965,-0.097966,NaN
2,charing cross london uk,en,NaN,ChIJrStISs4EdkgRvEoozoLVlx0,"Charing Cross, London, UK",neighborhood,United Kingdom,GB,England,England,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,51.509028,-0.125502,NaN
3,the london library 14 st james s square london uk,en,NaN,ChIJNZJ_3tYEdkgRWe8bC0b2gdI,"14 St James's Square, St. James's, London SW1Y...",establishment,United Kingdom,GB,England,England,...,Saint James's Square,SW1Y 4LG,NaN,NaN,NaN,NaN,NaN,51.507408,-0.136591,NaN
4,aldgate east station london uk,en,NaN,ChIJNaccT7UcdkgRCnzTgAKK8k0,"Aldgate East Station, Shadwell, London E1 1FE, UK",establishment,United Kingdom,GB,England,England,...,NaN,E1 1FE,NaN,NaN,NaN,NaN,NaN,51.515562,-0.071242,NaN


### Sequences, naively

For each volume sequence, look up each place, get lat/lon, calculate distance from previous place, sum over sequential path.

Issues:

* Allows the Boston -> USA distance
* Lookup on `name` field in GeoNames is brittle

Should match Sandeep's distance calculations.

In [156]:
gn.sample(1)

,name,asciiname,alternatenames,lat,lon,feature_class,feature_code,country_code,cc2,admin1_code,admin2_code,admin3_code,admin4_code,population,elevation,dem,timezone,mod_date
geonameid,,,,,,,,,,,,,,,,,,
1103108,Lago Nhanongo,Lago Nhanongo,,-23.90444,33.62667,H,LK,MZ,,3.0,,,,0,,53,Africa/Maputo,1998-12-08


In [157]:
gn.loc[1103108, ['lat', 'lon']]

lat   -23.90444
lon    33.62667
Name: 1103108, dtype: object

In [158]:
wi.loc[90, ['lat', 'lon']]

lat    51.507351
lon    -0.127758
Name: 90, dtype: object

In [178]:
def get_lat_lon(place_string, geo_data, source='geonames'):
    if source=='geonames':
        try:
            idx = geo_data.loc[geo_data.name==place_string, 'population'].idxmax()
        except ValueError:
            return(None)
    if source=='wilkens':
        try:
            idx = geo_data.loc[geo_data.text_string==place_string.lower(), 'occurs'].idxmax()
        except ValueError:
            return(None)
    return(geo_data.loc[idx, ['lat', 'lon']])

valid_names_gn = set(gn.name)
valid_names_wi = set(wi.text_string)

def sequence_distance(sequence, source='geonames'):
    # remove unknown locations
    if source=='geonames':
        valid_names = valid_names_gn
        geo_data = gn
        seq = [i for i in sequence if i in valid_names]
    if source=='wilkens':
        valid_names = valid_names_wi
        geo_data = wi
        seq = [i.lower() for i in sequence if i.lower() in valid_names]
    total_distance = 0.0
    for location_pair in pairwise(seq):
        loc1 = get_lat_lon(location_pair[0], geo_data, source=source)
        loc2 = get_lat_lon(location_pair[1], geo_data, source=source)
        if loc1 is None or loc2 is None:
            pass
        else:
            total_distance += distance.distance(loc1, loc2).miles
    return(total_distance)

In [176]:
df.head().loc[:, 'gpe_sequences'].apply(sequence_distance, source='geonames')

book_id
[Heist Society 2] Uncommon Criminals - Ally Carter     62668.457945
2001_2011_Wilson,RobertCharles_TheChronoliths_SF      135236.286029
2001_Martel,Yann_LifeofPi_BS                          144735.896805
2002_2011_Anderson,MT_Feed_SF                          11909.130762
2002_Baker,Jo_Offcomer_CT                              22038.499108
Name: gpe_sequences, dtype: float64

In [179]:
df.head().loc[:, 'gpe_sequences'].apply(sequence_distance, source='wilkens')

book_id
[Heist Society 2] Uncommon Criminals - Ally Carter     53271.747972
2001_2011_Wilson,RobertCharles_TheChronoliths_SF      162902.400686
2001_Martel,Yann_LifeofPi_BS                          199572.653692
2002_2011_Anderson,MT_Feed_SF                           4993.077876
2002_Baker,Jo_Offcomer_CT                              18895.382501
Name: gpe_sequences, dtype: float64

In [136]:
df['dist_miles'].head()

book_id
[Heist Society 2] Uncommon Criminals - Ally Carter     53272.047166
2001_2011_Wilson,RobertCharles_TheChronoliths_SF      179494.187170
2001_Martel,Yann_LifeofPi_BS                          199572.653692
2002_2011_Anderson,MT_Feed_SF                           4993.077876
2002_Baker,Jo_Offcomer_CT                              18895.382501
Name: dist_miles, dtype: float64

To do:

* Full prepcrossing of strings for my version of the geo data
* Duplicate removal for geonames: for all dupes in `name`, retain only the one with largest population. This will speed up the lookups a lot.

### Hop mobility